In [25]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gensim
import multiprocessing
from keras.preprocessing.sequence import pad_sequences

In [26]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [27]:
train.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [28]:
# Remove all non-letter characters and make everything lowercase.
train['comment_text'] = train['comment_text'].str.replace('[^a-zA-Z]',' ').str.lower()
test['comment_text'] = test['comment_text'].str.replace('[^a-zA-Z]',' ').str.lower()

In [29]:
train['comment_text'].head(10)

0    explanation why the edits made under my userna...
1    d aww  he matches this background colour i m s...
2    hey man  i m really not trying to edit war  it...
3      more i can t make any real suggestions on im...
4    you  sir  are my hero  any chance you remember...
5       congratulations from me as well  use the to...
6         cocksucker before you piss around on my work
7    your vandalism to the matt shirvington article...
8    sorry if the word  nonsense  was offensive to ...
9    alignment on this subject and which are contra...
Name: comment_text, dtype: object

In [30]:
train_split = train['comment_text'].str.split()
test_split = test['comment_text'].str.split()

In [31]:
train_lol = train_split.values.tolist()
test_lol = test_split.values.tolist()

In [32]:
# Gensim has support for multi-core systems
multiprocessing.cpu_count()

4

In [62]:
w2v = gensim.models.word2vec.Word2Vec(sentences=train_lol,workers=4,size=300)

In [63]:
from collections import defaultdict
vocab = defaultdict(int)
for k, v in w2v.wv.vocab.items():
    vocab[k]=v.index

In [64]:
max([v for k, v in w2v.wv.vocab.items()])

In [65]:
max([v for k,v in vocab.items()])

41526

In [66]:
train_ind = [[vocab[w] for w in train_split[i]] for i in range(len(train_split))]


In [67]:
print (train_ind[:10])

[[660, 76, 0, 129, 132, 178, 30, 648, 4288, 10991, 996, 86, 328, 48, 2148, 21, 11154, 53, 6526, 14, 61, 2634, 150, 2, 2813, 35, 120, 1155, 14992, 2674, 4, 46, 59, 21, 241, 0, 357, 32, 0, 37, 29, 146, 2, 68, 3228, 89], [144, 16357, 50, 2511, 12, 543, 3615, 2, 68, 4322, 2597, 22, 95, 37, 943, 182], [401, 411, 2, 68, 136, 13, 247, 1, 75, 308, 9, 17, 53, 8, 12, 582, 7, 2194, 485, 496, 104, 4, 591, 1, 38, 324, 129, 356, 3, 30, 37, 29, 50, 210, 1, 424, 60, 39, 0, 2281, 94, 0, 709, 461], [60, 2, 36, 21, 98, 56, 325, 1386, 14, 2016, 2, 5725, 23, 0, 124, 2380, 58, 15, 504, 14, 26, 6, 4034, 3, 1262, 3, 9214, 2, 69, 0, 273, 85, 125, 12844, 40, 8, 48, 19, 43, 10, 0, 1409, 140, 1203, 2008, 413, 1203, 316, 2, 36, 34, 8, 504, 14, 23, 47, 51, 288, 105, 116, 23, 5, 18, 56, 4161, 11, 2281, 271, 14, 273, 26, 107, 1, 34, 9, 208, 46, 204, 38, 71, 42, 533, 1, 15, 6, 5635, 14, 80, 11, 376, 40, 2, 617, 42, 85, 15, 6, 1937, 379, 6, 2809, 3098, 77, 9, 17, 414, 10, 0, 496, 614, 3413, 28, 99, 24, 3384, 4623], [5,

In [68]:
test_ind = [[vocab[w] for w in test_split[i]] for i in range(len(test_split))]

In [69]:
len(train_ind)

159571

In [70]:
train_padded = pad_sequences(train_ind,maxlen=200,truncating='pre')
test_padded = pad_sequences(test_ind,maxlen=200,truncating='pre')

In [71]:
print (len(train_padded))
print (len(test_padded))


159571
153164


In [72]:
train_padded.shape

(159571, 200)

In [73]:
train_padded

array([[    0,     0,     0, ...,    68,  3228,    89],
       [    0,     0,     0, ...,    37,   943,   182],
       [    0,     0,     0, ...,     0,   709,   461],
       ...,
       [    0,     0,     0, ...,  3297, 12587,  4190],
       [    0,     0,     0, ...,   153,    35,     9],
       [    0,     0,     0, ...,  1561,  1964,    88]])

In [74]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional, GlobalMaxPool1D, BatchNormalization,Embedding
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, GRU, Embedding, Dropout, Activation, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D

from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
embed_size = 50 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 200 # max number of words in a comment to use

In [75]:
emb_layer = w2v.wv.get_keras_embedding()

In [76]:
emb_layer

In [77]:
inp = Input(shape=(maxlen,))
x = emb_layer(inp)

x = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPooling1D(pool_size=2)(x)

x = Bidirectional(LSTM(50, return_sequences=True,dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = BatchNormalization()(x)
x = Dense(50, activation="relu")(x)
#x = BatchNormalization()(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="softmax")(x)

model = Model(inputs=inp, outputs=x)

print(model.summary())
import keras.backend as K
def loss(y_true, y_pred):
     return K.binary_crossentropy(y_true, y_pred)
    
model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 100, 300)          12458100  
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 100, 32)           28832     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 50, 32)            0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 50, 100)           33200     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 100)               0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 100)               400       
__________

In [55]:
y = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]

In [56]:
from keras import callbacks
def schedule(ind):
    a = [0.002,0.003, 0.000]
    return a[ind]
lr = callbacks.LearningRateScheduler(schedule)

model.fit(x=train_padded,y=y,validation_split=.1,epochs=3,batch_size=64)


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 462s 3ms/step - loss: 0.2976 - acc: 0.9331 - val_loss: 0.2884 - val_acc: 0.9873 - ETA: 1s - loss: 0.2978 - ac
Epoch 2/3
143613/143613 [==============================] - 423s 3ms/step - loss: 0.2834 - acc: 0.9818 - val_loss: 0.2843 - val_acc: 0.9842
Epoch 3/3
143613/143613 [==============================] - 435s 3ms/step - loss: 0.2808 - acc: 0.9816 - val_loss: 0.2852 - val_acc: 0.9855


In [70]:
a=[144, 16196, 50, 2511, 12, 543, 3617, 2, 68, 4321, 2599, 22, 95, 37, 944, 182]

In [71]:
test_pred = model.predict(x=a)
y_classes = test_pred.argmax(axis=-1)
#y_classes = keras.np_utils.probas_to_classes(test_pred)
#This is equivalent to model.predict_classes(x) on the Sequential model.

ValueError: Error when checking : expected input_8 to have shape (100,) but got array with shape (1,)

In [61]:
s=set(y_classes)
print(s)
print(len(y_classes))
print(y_classes.shape)

{0, 1, 2, 3, 4, 5}
153164
(153164,)


In [35]:
print(test_pred)

[[9.9835712e-01 3.0494604e-01 9.7802311e-01 4.6374734e-02 9.3047005e-01
  3.4360987e-01]
 [1.7355323e-04 1.7877742e-10 1.8291954e-05 1.9762727e-09 5.5348914e-06
  3.7253318e-07]
 [1.1260275e-02 4.8330803e-06 5.3420700e-03 4.6667701e-06 1.5321537e-03
  1.1742234e-04]
 ...
 [2.6474227e-05 1.7033279e-10 1.3006506e-06 1.5546626e-09 7.0798382e-07
  1.7949057e-07]
 [3.3716680e-04 6.0020696e-08 6.6738885e-06 2.6853630e-07 7.1242816e-06
  6.0562029e-06]
 [8.0460435e-01 7.7419532e-03 5.2338749e-01 8.1460609e-04 2.1585009e-01
  2.9234942e-03]]
